In [1]:
from datasets import load_dataset
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from transformers import BartForConditionalGeneration, BartTokenizer, TrainingArguments
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import threading
import time
from threading import Thread
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
import math
import random
import chromadb
from chromadb.config import Settings
chroma_client = chromadb.PersistentClient(path="./")

C:\Users\Akshay\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import findspark
findspark.init()

In [3]:
# spark = SparkSession.builder \
#     .master("local") \
#     .config("spark.driver.memory", "2g") \
#     .config("spark.executor.memory", "2g") \
#     .appName("test") \
#     .getOrCreate()
# df = spark.createDataFrame([(1, 'Alice'), (2, 'Bob')], ['id', 'name'])
# df.show()

### Context Based Dataset

In [29]:
dataset_1 = load_dataset("9wimu9/eli5_mult_answers_en_no_answer_in_context") # in deep
dataset_2 = load_dataset("mlxen/squad_1_1_smallcase_context") #one word
dataset_3 = load_dataset("nbtpj/multi-context-long-answer-dataset") # in short
dataset_8 = load_dataset("LasRuinasCirculares/sft_correct")
dataset_9 = load_dataset("robbiegwaldd/rephrase_train") # all
ds = load_dataset("addy88/nq-question-answeronly")
ds = load_dataset("iarfmoose/question_generator")

In [30]:
dataset_1

DatasetDict({
    train: Dataset({
        features: ['question', 'contexts', 'gold_answer'],
        num_rows: 71236
    })
    test: Dataset({
        features: ['question', 'contexts', 'gold_answer'],
        num_rows: 7916
    })
})

In [31]:
agent_1 = dataset_1["train"]["question"]
raw_context = dataset_1["train"]["contexts"]
modified_context = []
for c in raw_context:
    final_string = ""
    for i in c:
        final_string = final_string+" "+ i
    modified_context.append(final_string)
context = modified_context
agent_2 = dataset_1["train"]["gold_answer"]

In [32]:
agent_1.extend(dataset_2["train"]["question"])
context.extend(dataset_2["train"]["context"])
agent_2.extend([i["text"][0] for i in dataset_2["train"]["answers"]])

agent_1.extend([i.split("Question:")[1].replace("Answer:","") for i in dataset_8["train"]["instruction"]])
context.extend([i.replace("Evidence:","").replace("**","").split("Question:")[0].replace("\n","") for i in dataset_8["train"]["instruction"]])
agent_2.extend(dataset_8["train"]["output"])

In [33]:
channelNames =["pubmed_qa_X_squad_num_channel_1_test","pubmed_qa_X_squad_num_channel_1_train","pubmed_qa_X_squad_num_channel_2_test","pubmed_qa_X_squad_num_channel_2_train","pubmed_qa_X_squad_num_channel_3_test","pubmed_qa_X_squad_num_channel_3_train","pubmed_qa_X_squad_num_channel_4_test","pubmed_qa_X_squad_num_channel_4_train","pubmed_qa_num_channel_1_test","pubmed_qa_num_channel_1_train","pubmed_qa_num_channel_2_test","pubmed_qa_num_channel_2_train","pubmed_qa_num_channel_3_test","pubmed_qa_num_channel_3_train","pubmed_qa_num_channel_4_test","pubmed_qa_num_channel_4_train","squad_num_channel_1_test","squad_num_channel_1_train","squad_num_channel_2_test","squad_num_channel_2_train","squad_num_channel_3_test","squad_num_channel_3_train","squad_num_channel_4_test","squad_num_channel_4_train"]

In [34]:
question = []
cont = []
answer = []
for channel in channelNames:
    question.extend([i[0].split("<||||>")[0] for i in dataset_3[channel]["context"]])
    cont.extend([i[0].split("<||||>")[1] for i in dataset_3[channel]["context"]])
    answer.extend([i for i in dataset_3[channel]["answer"]])
temp_df = pd.DataFrame()
temp_df["question"] = question
temp_df["cont"] = cont
temp_df["answer"] = answer
df = temp_df.drop_duplicates(subset=['question','cont','answer'], keep='first')
agent_1.extend(df["question"].tolist())
context.extend(df["cont"].tolist())
agent_2.extend(df["answer"].tolist())

In [35]:
datasets = ["bart-base-squad.end2end.amazon","bart-base-squad.end2end.new_wiki","bart-base-squad.end2end.nyt","bart-base-squad.end2end.reddit","bart-base-squad.multitask.amazon","bart-base-squad.multitask.new_wiki","bart-base-squad.multitask.nyt","bart-base-squad.multitask.reddit","bart-base-squad.pipeline.amazon","bart-base-squad.pipeline.new_wiki","bart-base-squad.pipeline.nyt","bart-base-squad.pipeline.reddit","bart-base-squad.qg_reference.amazon","bart-base-squad.qg_reference.new_wiki","bart-base-squad.qg_reference.nyt","bart-base-squad.qg_reference.reddit","bart-large-squad.end2end.amazon","bart-large-squad.end2end.new_wiki","bart-large-squad.end2end.nyt","bart-large-squad.end2end.reddit","bart-large-squad.multitask.amazon","bart-large-squad.multitask.new_wiki","bart-large-squad.multitask.nyt","bart-large-squad.multitask.reddit","bart-large-squad.pipeline.amazon","bart-large-squad.pipeline.new_wiki","bart-large-squad.pipeline.nyt","bart-large-squad.pipeline.reddit","bart-large-squad.qg_reference.amazon","bart-large-squad.qg_reference.new_wiki","bart-large-squad.qg_reference.nyt","bart-large-squad.qg_reference.reddit","t5-base-squad.end2end.amazon","t5-base-squad.end2end.new_wiki","t5-base-squad.end2end.nyt","t5-base-squad.end2end.reddit","t5-base-squad.multitask.amazon","t5-base-squad.multitask.new_wiki","t5-base-squad.multitask.nyt","t5-base-squad.multitask.reddit","t5-base-squad.pipeline.amazon","t5-base-squad.pipeline.new_wiki","t5-base-squad.pipeline.nyt","t5-base-squad.pipeline.reddit","t5-base-squad.qg_reference.amazon","t5-base-squad.qg_reference.new_wiki","t5-base-squad.qg_reference.nyt","t5-base-squad.qg_reference.reddit","t5-large-squad.end2end.amazon","t5-large-squad.end2end.new_wiki"]

In [36]:
for i in tqdm(range(0,len(datasets))):
    # print(f'\rDataset Name: {dataset}', end='', flush=True)
    ds = load_dataset("lmqg/qa_squadshifts_synthetic", datasets[i])
    agent_1.extend(i for i in ds["train"]["question"])
    context.extend(i for i in ds["train"]["context"])
    agent_2.extend(i["text"][0] for i in ds["train"]["answers"])

100%|██████████| 50/50 [01:00<00:00,  1.21s/it]


In [37]:
dataset_9

DatasetDict({
    train: Dataset({
        features: ['id', 'original_snippet', 'rephrased_snippet'],
        num_rows: 291032
    })
})

In [38]:
len(agent_1)

1469571

In [39]:
questions = []
answers = []
conte = []
split_qa = [i.split("\n\n") for i in dataset_9["train"]["rephrased_snippet"]]
cont = [i.replace("\n","") for i in dataset_9["train"]["original_snippet"]]
for i in tqdm(range(0,len(split_qa))):
    q = [];a = [];c = []
    if "\nAnswer" not in split_qa[i][0]:
        if len(split_qa[i]) % 2 != 0:
            split_qa[i] = split_qa[i][:-1]
        for j in range(0,len(split_qa[i]),2):
            q.append(split_qa[i][j])
            a.append(split_qa[i][j+1])
            c.append(cont[i])
    else:
        new_split = [j.split("\n") for j in split_qa[i]]
        for j in range(0,len(new_split)):
            if len(new_split[j]) %2 !=0:
                continue
            q.append(new_split[j][0].replace("Question:",""))
            a.append(new_split[j][1].replace("Answer:",""))
            c.append(cont[i])
    answers.extend(a)
    questions.extend(q)
    conte.extend(c)
    # print(len(answers),len(questions))
    # print(split_qa[i])
agent_1.extend(questions)
context.extend(conte)
agent_2.extend(answers)

100%|██████████| 291032/291032 [00:00<00:00, 369566.84it/s]


In [40]:
len(agent_1)

1802143

In [41]:
len(context)

1802143

In [42]:
len(agent_2)

1802143

In [43]:
temp_df = pd.DataFrame()
temp_df["agent_1"] = agent_1
temp_df["context"] = context
temp_df["agent_2"] = agent_2

In [5]:
temp_df.to_csv("../Data/CleanedDatasets/ContextBasedQuestions.csv")

In [16]:
temp_df = pd.read_csv("../Data/CleanedDatasets/ContextBasedQuestions.csv")

In [40]:
temp_df.head()

,agent_2,agent_1
0,agent_2:These foods contain certain sugars tha...,<context> Actually it is because most of peop...
1,agent_2:I'd answer the question but the defaul...,<context> The decay of false vacuum is a fasc...
2,agent_2:Essentially the events in Crimea threa...,<context> National Sovereignty is a very big ...
3,agent_2:To have as little protection as is saf...,<context> Most helmets are pretty similar in ...
4,"agent_2:Your eyes are magnificent sensors, and...",<context> The reason is that a camera is basi...


In [18]:
temp_df["agent_2"] = "agent_2:"+temp_df["agent_2"]
temp_df["agent_1"] = "<context> " + temp_df["context"] + "agent_1:"+temp_df["old_agent_1"]

In [19]:
temp_df.dropna(inplace = True)
temp_df.reset_index(inplace=True)

In [36]:
import math
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
agent_1_extended = []
agent_2_extended = []
tokenizedLengthList = []
idx = 0
for input_text in tqdm(temp_df["agent_1"].to_list()):
    try:
        input_tokenized = tokenizer(input_text,return_tensors='pt')
        tokenizedLength = len(input_tokenized["input_ids"][0])
        tokenizedLengthList.append(tokenizedLength)
        if tokenizedLength < 510:
            agent_1_extended.append(input_text)
            agent_2_extended.append(temp_df["agent_2"][idx])
    except Exception as e:
        print(e)
    idx = idx+1
    # if tokenizedLength > 510 and tokenizedLength < 1500:
        
    #         num_of_loops = math.ceil(tokenizedLength/510)
    #         slice_length = math.ceil(len(input_text)/num_of_loops)
    #         for i in range(0,num_of_loops):
    #             if i == 0:
    #                 agent_1_extended.append("agent_1:"+(input_text[i*slice_length:(i*slice_length)+slice_length])+"<question>"+temp_df["old_agent_1"][idx])
    #             else:
    #                 agent_1_extended.append("agent_1:"+"<context>"+(input_text[(i*slice_length)-50:(i*slice_length)+slice_length])+"<question>"+temp_df["old_agent_1"][idx])
    
    # else:
    #     agent_1_extended.append("agent_1:"+input_text)

100%|██████████| 1799426/1799426 [45:44<00:00, 655.72it/s] 


In [37]:
len(agent_1_extended)

1400831

In [38]:
len(agent_2_extended)

1400831

In [39]:
temp_df = pd.DataFrame()
temp_df["agent_2"] =agent_2_extended
temp_df["agent_1"] =agent_1_extended

In [11]:
agent_1_extended[2]

'<context>  National Sovereignty is a very big deal so anytime a country (especially a country with nukes) starts to reject previously agreed upon borders it is a big deal.\n\nThe reason you should care is that Russia has been for quite sometime trying to maintain some degree of influence over several former Soviet republics.  As some of these states move away from Russia, it is going to cause conflict and potentially a full blown war. likely to evolve into nothing\n\ncrimea was granted to ukraine as a restitution gift and well over half the population is in favour of annexation\n\nthey already speak russian and maintain russian culture\n\nputin is a total dick but once this blows over they will pay lower taxes to an equally corrupt government and receive additional social benefits\n\nyeah, its a huge problem that a part of a country is essentially being taken over but it hardly sets precedent for speculations of a third world war\n\nagent_1:what is the big deal about russia invading u

In [55]:
temp_df.to_csv("../Data/CleanedDatasets/ContextBasedQuestions.csv")

In [53]:
temp_df["agent_1"][1400700]

"<context> 5 best Java seeds for Minecraft 1.19 updateThe 1.19 update for Minecraft introduced many new features. \xa0Two major biomes, Deep Dark and Mangrove Swamp, have been added to the game and are quickly becoming quite popular.The two biomes are arguably the biggest addition to the game. \xa0Deep Dark and Mangrove Swamp both provide reasons to visit them, but there's only one problem: they can be difficult to find.Minecraft seeds to try in Java Edition version 1.19Multiple Blue Rings\xa0Note: The 1.18 update introduced seed universality, so seeds will work on both versions with minimal differences. \xa0The main differences are when moving from Java to Bedrock, but not vice versa.The ancient city in 1.19 is a new but extremely rare structure. \xa0It's a challenge to trace a regular Deep Dark with just the Skulk and the Warden. \xa0Most of those found do not even contain any ancient cities.Seed: -457009213479927390This seed puts players into an amazing spawn. \xa0Gamers come right 

In [54]:
temp_df["agent_2"][1400700]

'agent_2: In Minecraft 1.19, there are two new biomes added: "Deep Dark" and "Mangrove Swamp". The Deep Dark biome provides reasons to visit it but can be difficult to find due to its location near a swamp. Meanwhile, the Mangrove Swamp biome has only one problem - it can be difficult to find as well. Minecraft seeds like -457009213479927390, -6709148406763899126, and -2110863992403414331 can be used to trace a Deep Dark with just the Skulk and Warden. These seeds contain good loot but do not necessarily have Mangrove Swamp as they are located in swamps instead of in the swamp biome. The seeds -2110863992403414331 and 1450778142214593647 can be used to find Mangrove Swamp by using a village as an alternative spawn point. The seeds -6709148406763899126 and 6705'

### Google Search

In [4]:
import torch
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = '../Saved_Models/Sky/fine-tuned-bert-sentiment_2024_10_05_0'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
model = model.to(device)
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

#### Normal Search Queries

In [5]:
query = "Reinforcement learning with human feedback?"	

In [6]:
from googlesearch import search
def get_google_search_links(query):
    return [link for link in search(query)]

In [7]:
import re

def clean_latex(text):
    # Remove LaTeX commands like {\\displaystyle}, {\\text{}}, etc.
    cleaned_text = re.sub(r'\\displaystyle|\\text\{.*?\}', '', text)
    
    # Remove any LaTeX curly braces and unnecessary whitespaces
    cleaned_text = re.sub(r'\\[a-z]+|{|}', '', cleaned_text)
    
    # Replace LaTeX-specific representations like \\dots with their equivalent
    cleaned_text = re.sub(r'\\dots', '...', cleaned_text)
    
    # Remove multiple spaces introduced by LaTeX removal
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

In [8]:
website_links = get_google_search_links(query)

In [9]:
website_links

['https://arxiv.org/abs/2201.08102',
 'https://en.wikipedia.org/wiki/Reinforcement_learning_from_human_feedback',
 'https://thisisrishi.medium.com/reinforcement-learning-with-human-feedback-in-llms-a-comprehensive-guide-771b381e94e7',
 'https://encord.com/blog/guide-to-rlhf/',
 'https://arxiv.org/abs/1709.03969',
 'https://huggingface.co/learn/deep-rl-course/en/unitbonus3/rlhf',
 'https://manikanthgoud123.medium.com/what-is-rlhf-reinforcement-learning-from-human-feedback-d0ec88e0866c',
 'https://annotationbox.com/reinforcement-learning-from-human-feedback/',
 'https://bdtechtalks.com/2023/01/16/what-is-rlhf/',
 'https://www.leewayhertz.com/reinforcement-learning-from-human-feedback/',
 'https://aws.amazon.com/what-is/reinforcement-learning-from-human-feedback/']

In [10]:
class ThreadWithReturnValue(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None

    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return

In [11]:
def data_mining(website):
    filtered_content = ""
    if (".gov" not in website) or ("linkedin.com" not in website) or ("reddit.com" not in website):
        URL = website
        r = requests.get(URL) 
        soup = BeautifulSoup(r.content, 'html5lib')
        for tag in soup(['nav', 'header', 'footer', 'script', 'style', 'aside']):
            tag.decompose()
        for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'p', 'li','strong']):
            filtered_content = filtered_content+tag.get_text()
    return filtered_content


In [14]:
def create_chunks(cleaned_data):
    ids_list = []
    final_chunks = []
    random_number = random.randint(0,10000000000)
    loop = math.ceil(len(cleaned_data)/2000)
    for i in range(0,loop):
        if i ==0:
            final_chunks.append(cleaned_data[(i*2000):(i+1)*2000])
        else:
            final_chunks.append(cleaned_data[(i*2000)-500:(i+1)*2000])
        ids_list.append(str(random_number+(i/50)))
    return (final_chunks,ids_list)
    # return final_chunks
    

In [41]:
from pyspark.sql import functions as F
start_time = time.time()
# spark = SparkSession.builder.appName("WebScrapingWithPySpark").getOrCreate()
threads_list = [ThreadWithReturnValue(target=data_mining, args=(website,)) for website in website_links]
[thread.start() for thread in threads_list]
fetched_data = [thread.join() for thread in threads_list]

cleaned_data_udf = udf(clean_latex, StringType())
# create_chunks_udf = udf(create_chunks, ArrayType(StringType()), ArrayType(StringType()))
schema = StructType([
    StructField("chunks", ArrayType(StringType())),
    StructField("ids", ArrayType(StringType()))
])
create_chunks_udf = udf(create_chunks, schema)

spark = SparkSession.builder \
    .master("local") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .appName("WebScrapingWithPySpark") \
    .getOrCreate()

temp_df = list(zip(website_links, fetched_data))
uncleaned_df = spark.createDataFrame(temp_df, ["url", "uncleaned_data"])
cleaned_df = uncleaned_df.withColumn("cleaned_data", cleaned_data_udf("uncleaned_data"))
clean_df = cleaned_df.withColumn("processed", create_chunks_udf("cleaned_data"))

clean_df = clean_df.select(
    F.col('processed.chunks').getItem(0).alias("chunks"),
    F.col('processed.ids').getItem(1).alias("ids")
)

clean_df.show()
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

+--------------------+-------------+
|              chunks|          ids|
+--------------------+-------------+
|Computer Science ...|9962182769.02|
|Supervised learni...| 678886517.02|
|Sign upSign inSig...|7893596736.02|
|Please enable Jav...|9379303736.02|
|Computer Science ...|         NULL|
|Deep RL Course do...|9585043200.02|
|Sign upSign inSig...|6576366173.02|
|You have reached ...|         NULL|
|HomeBlogTips & Tr...|5751933141.02|
|Reinforcement Lea...| 606611957.02|
|What is Cloud Com...|2717412571.02|
+--------------------+-------------+

Execution time: 2.327502965927124 seconds


In [43]:
rows = clean_df.collect()

# Access the nth row (for example, 3rd row, index 2)
# nth_row = rows[4]
# nth_row['processed'][0]

In [44]:
rows

[Row(chunks='Computer Science > Machine LearningarXiv:2201.08102Title:Safe Deep RL in 3D Environments using Human FeedbackSubmission history[v1][v2]Access Paper:View PDFTeX SourceOther FormatsAncillary files (details):Cliff edge environment/images/image1.mp4Cliff edge environment/images/image10.mp4Cliff edge environment/images/image11.mp4Cliff edge environment/images/image12.mp4Cliff edge environment/images/image2.mp4Cliff edge environment/images/image3.mp4Cliff edge environment/images/image4.pngCliff edge environment/images/image5.mp4Cliff edge environment/images/image6.mp4Cliff edge environment/images/image7.pngCliff edge environment/images/image8.pngCliff edge environment/images/image9.mp4Cliff edge environment/instructions.htmlDangerous blocks environment/images/image1.mp4Dangerous blocks environment/images/image10.mp4Dangerous blocks environment/images/image11.pngDangerous blocks environment/images/image12.mp4Dangerous blocks environment/images/image2.pngDangerous blocks environme

In [47]:
rows[2]

Row(chunks='Sign upSign inSign upSign inReinforcement Learning with Human Feedback in LLMs: A Comprehensive GuideRishiFollow--1ListenShareIntroductionLarge Language Models (LLMs) are everywhere these days, helping us process and understand natural language. They have the potential to change the game in various industries and make human-computer interactions even better. However, we need to make sure they align with human values and generate reliable and diverse outputs. To achieve this, we use Reinforcement Learning with Human Feedback (RLHF) to train LLMs and incorporate human preferences. This technique is crucial to improving their safety and helpfulness.In this guide, we’ll take a deep dive into an important process called RLHF and its role in developing modern LLMs. We’ll walk through the steps involved in RLHF, compare how it’s used in popular models like ChatGPT and Llama 2, and talk about the alternatives and limitations of RLHF. By the end of this guide, you’ll have a clear un

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 57883)
Traceback (most recent call last):
  File "C:\Users\Akshay\anaconda3\envs\tf\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\Akshay\anaconda3\envs\tf\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\Akshay\anaconda3\envs\tf\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\Akshay\anaconda3\envs\tf\lib\socketserver.py", line 747, in __init__
    self.handle()
  File "C:\Users\Akshay\anaconda3\envs\tf\lib\site-packages\pyspark\accumulators.py", line 295, in handle
    poll(accum_updates)
  File "C:\Users\Akshay\anaconda3\envs\tf\lib\site-packages\pyspark\accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "C:\Users\Akshay\anaconda

In [40]:
for r in rows:
    print(len(r[3][0]),len(r[3][1]))

2 2
19 19
4 4
8 8
1 1
3 3
5 5
1 1
8 8
29 29
5 5


In [24]:
# Stop Spark session1.92173671
# spark.stop()

In [18]:
collection = chroma_client.get_or_create_collection(name="my_collection")

In [19]:
# chroma_client.delete_collection(name="my_collection")

In [20]:
collection.count()

804

In [21]:
collection.upsert(
    documents=final_chunks,
    ids=ids_list
)

In [22]:
start_time = time.time()

results = collection.query(
    query_texts=[query], # Chroma will embed this for you
    n_results=2 # how many results to return
)
# print(results)

end_time = time.time()

# Calculate the total time taken
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Execution time: 0.07861828804016113 seconds


In [23]:
results['distances']

[[0.7082719453848038, 0.7667292356491089]]

In [24]:
input_ids = tokenizer.encode("<context>{}agent_1:{}".format(results["documents"][0][0],query), return_tensors='pt',max_length=1024, truncation=True).to(device)
output = model.generate(
    input_ids, 
    max_length=512, 
    num_beams=10, 
    early_stopping=True, 
    no_repeat_ngram_size=2,  # Prevent repeating n-grams
    num_return_sequences=1,  # Number of sequences to return
    temperature=0.7,  # Sampling temperature
    top_k=50,  # Top-K sampling
    top_p=0.9  # Top-p (nucleus) sampling
)
resp = tokenizer.decode(output[0], skip_special_tokens=True)
resp

'agent_2:Yes, he has a Canadian passport.'

In [18]:
from torch.nn.functional import softmax

# Tokenize the input
input_ids = tokenizer.encode("<context>{}agent_1:{}".format(results["documents"][0][0],query), return_tensors='pt', max_length=512, truncation=True).to(device)

# Forward pass through the model to get the logits
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)
    logits = outputs.logits

# Get the predicted token ids from the logits (the most likely tokens)
predicted_ids = torch.argmax(logits, dim=-1)

# Calculate the probabilities (softmax) for the predicted tokens
probabilities = softmax(logits, dim=-1)

# Get the probabilities of the predicted tokens
predicted_probabilities = probabilities.gather(2, predicted_ids.unsqueeze(-1)).squeeze(-1)

# Calculate the confidence score as the average probability of the predicted tokens
confidence_score = predicted_probabilities.mean().item()

print("Generated Output:", tokenizer.decode(predicted_ids[0], skip_special_tokens=True).replace("\n",""))
print("Confidence Score:", confidence_score)

Generated Output: agentagent!-->omes paradise, swanky Mauritius bungalow, and a stunning villa in Goa. how Akshay transformed from a humble beginning to owning some of the world’s most luxurious homes. With exclusive insights into his glamorous residences and the stunning interiors crafted by his wife Twinkle Khanna, get ready to be amazed by the opulence and grandeur that define the Khiladi’s lifestyle! of Akshay Kumar & Home Address of Akshay Kumar was Home Address Akshay Kumar was born on the 9th of September 1967. He was born in Amritsar, Punjab, India and moved to Delhi where he spent most of his childhood. When his father retired from the army to join UNICEF he moved to Mumbai. Rajiv Hari Om Bhatia made his way into Hindi cinema in the early 1990s, after a series of flops came the breakthrough role in Khiladi directed by Abas-Mastan established Akshay Kumar as an action hero. Akshay Kumar adopted the professional name of Akshay Kumar before stepping into movies. His charming smil

# Vector Database

In [19]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('chroma.sqlite3') 

# Create a cursor object
cursor = conn.cursor()

In [20]:
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
cursor.execute(sql_query)
print(cursor.fetchall())

[('migrations',), ('embeddings_queue',), ('embeddings_queue_config',), ('collection_metadata',), ('segment_metadata',), ('tenants',), ('databases',), ('collections',), ('maintenance_log',), ('segments',), ('embeddings',), ('embedding_metadata',), ('max_seq_id',), ('embedding_fulltext_search',), ('embedding_fulltext_search_data',), ('embedding_fulltext_search_idx',), ('embedding_fulltext_search_content',), ('embedding_fulltext_search_docsize',), ('embedding_fulltext_search_config',)]


In [21]:
sql_query = """SELECT * FROM collections;"""
cursor.execute(sql_query)

In [22]:
print(cursor.fetchall())

[('1c13d8d4-830b-4def-928f-fb4fc943daa2', 'my_collection', 384, '00000000-0000-0000-0000-000000000000', '{"hnsw_configuration": {"space": "l2", "ef_construction": 100, "ef_search": 10, "num_threads": 28, "M": 16, "resize_factor": 1.2, "batch_size": 100, "sync_threshold": 1000, "_type": "HNSWConfigurationInternal"}, "_type": "CollectionConfigurationInternal"}')]
